In [12]:
import pandas as pd
from dbfread import DBF
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import urllib.request

# Wind turbine optimization

## 0 - Importing data

In [20]:
"""
Load content of a DBF file into a Pandas data frame.

The iter() is required because Pandas doesn't detect that the DBF
object is iterable.

Loading the raw geospatial data into a Pandas data frame.

Datasource: https://www.swisstopo.admin.ch/en/home/meta/supply-structure/freely-available.html
"""

# Downloading a file from remote to a local file in a folder called Geospatial_raw_data_remote
dbf_url = 'https://github.com/percw/Swiss_wind-turbine_optimization/raw/b23d047c6f824bb225cc1b7d5aa559166f191ec6/Geospatial_raw_data/spat_pts/spat_pts.dbf'
shape_url = 'https://github.com/percw/Swiss_wind-turbine_optimization/raw/b23d047c6f824bb225cc1b7d5aa559166f191ec6/Geospatial_raw_data/spat_pts/spat_pts.shp'


# Checking if the folder Geospatial_raw_data exists, if not downloading the files from remote
if not os.path.exists('./Geospatial_raw_data/'):
	os.makedirs('./Geospatial_raw_data/spat_pts/')
	urllib.request.urlretrieve(dbf_url, './Geospatial_raw_data/spat_pts.dbf') 		# Loading the .dbf file
	urllib.request.urlretrieve(shape_url, './Geospatial_raw_data/spat_pts.shp') 	# Loading the .shp file

# Downloading the file from remote to local

# Loading the file from local to Pandas data frame
dbf = DBF('./Geospatial_raw_data/spat_pts.dbf', encoding='UTF-8')
#shp = DBF('./Geospatial_raw_data/spat_pts.shp', encoding='UTF-8')
df_dbf_raw = DataFrame(iter(dbf))
#df_shp_raw = DataFrame(iter(shp))



File: `spat_pts.dbf` - consists of candidate wind turbine locations in Switzerland which are technically feasible. The readme file provides details about the 30 attributes further attributes and the coordinate reference system. 

In [21]:
print(df_dbf_raw.head(10))
#print(df_shp_raw.head(10))

    PU_ID   LT  WT_ID  DIST_ISOS  DIST_UNE_N  DIST_UNE_K  DIST_BLN  DIST_VAEW  \
0   738.0  1.0    9.0      735.0    132448.0     16505.0     184.0   115789.0   
1   880.0  1.0   11.0      567.0    132177.0     16475.0     382.0   115340.0   
2   881.0  1.0   12.0      186.0    131844.0     15740.0     156.0   115343.0   
3  1021.0  1.0   15.0      424.0    131912.0     16340.0     389.0   114896.0   
4  1163.0  1.0   20.0     1288.0    131652.0     16255.0     349.0   114458.0   
5  1449.0  1.0   24.0      767.0    130084.0     14166.0     412.0   113543.0   
6  1586.0  1.0   25.0      940.0    131307.0     17251.0     650.0   113249.0   
7  1589.0  1.0   28.0     1856.0    130159.0     14700.0     223.0   113097.0   
8  1590.0  1.0   29.0      636.0    129811.0     14094.0     387.0   113089.0   
9  1591.0  1.0   30.0      847.0    129481.0     13388.0     225.0   113102.0   

   DIST_RAD  DIST_AIR  ...  COHER_AREA  VIS_ARE_ha  ENER_DENS     DIST_NATPA  \
0   26065.0    2861.0  ...  

## 0.1 - Discovering the data 

In [ ]:

print(f'Number of technically possible wind turbine locations: {len(df_raw["PU_ID"].unique())}')
print(f'Number of canton where its feasible to place wind turbine: {len(df_raw["CANT_NAME"].unique())}')
print(f'The cantons are: {df_raw["CANT_NAME"].unique()}')

Checking out the canton with the empty string

In [ ]:
display(df_raw[df_raw["CANT_NAME"] == ''])

## Watermark for programming environment

In [ ]:
%load_ext watermark
%watermark -v -m -p numpy,pandas,matplotlib,seaborn,dbfread,ortools
